## Actividad 2: Comparación de algoritmos (Clasificación)

### Importación de librerías
Importamos las librerías que se van a usar a lo largo del ejercicio

In [ ]:
# Manejo de datos
from pandas import read_csv,DataFrame
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

#Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix

#train-test
from sklearn.model_selection import train_test_split
from sklearn import model_selection

#métricas
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#Algoritmos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Descripción del dataset
'Para este ejercicio se usará el dataset **Prediction of music genre** extraído desde la página de Kaggle. El dataset cuenta con **más de 50 000 registros** de canciones y el objetivo es poder clasificar a qué genero musical pertenece

In [ ]:
#Leemos archivo desde una url de Kaggle
url_kaggle = (r'https://raw.githubusercontent.com/gad1989/TIA_actividad2/master/music_genre.csv')
datos = read_csv(url_kaggle,delimiter=",")

### Caracteristicas del dataset
Realizamos una descripcion del dataset. Primero damos un primer vistazo a algunos registros

In [ ]:
datos.head(5)

Con la función shape, comprobamos que cuenta con **50005 registros** y **18 atributos (incluyendo la clase)**

In [ ]:
datos.shape

Visualizamos los tipos de datos de cada variable, vemos que la mayoría son atributos numéricos

In [ ]:
datos.dtypes

Calculamos las principales estadísticas de los atributos numéricos

In [ ]:
datos.describe()

Revisamos los valores nulos en cada atributos, por lo visto hay 5 registros completamente vacíos los cuales deberán ser eliminados

In [ ]:
datos.isna().sum()

Observamos la distribución de la clase, la cual se encuentra perfectamente balanceada

In [ ]:
print(datos.groupby("music_genre").size())

Visualizamos las distribuciones de las variables continuas

In [ ]:
datos.hist(figsize=(16, 12))
plt.show()

Para el caso de las variables categóricas, mostramos la frecuencia de cada uno de sus valores

In [ ]:
datos.groupby("key").size()

In [ ]:
datos.groupby("mode").size()

In [ ]:
datos.groupby("tempo").size()

### Preparación de datos

Iniciamos con la limpieza de datos: se eliminan los 5 registros con valores nulos identificados en el paso anterior y también se eliminan columnas que claramente no aportarán en la clasificación **("instance_id","artist_name","track_name","obtained_date")**

In [ ]:
datos = datos.dropna()
datos = datos.drop(["instance_id","artist_name","track_name","obtained_date"],axis=1)

La variable **tempo** está mal catalogada como categórica ya que en realidad es numérica solo que existen un 10% de registros con valores faltantes que se han registrado como "?". Procedemos a imputar estos valores con el promedio del resto de registros y convertir la variable a numérica.

In [ ]:
#convierte valor "?" a nulo y luego castea la variable a float
datos.loc[datos['tempo'] == '?', 'tempo'] = np.nan
datos = datos.astype({'tempo': np.float64})

In [ ]:
prom_tempo = datos['tempo'].mean()
datos['tempo'] = datos['tempo'].fillna(prom_tempo)

Convertimos las variables **key y mode** a numéricas usando LabelEncoder

In [ ]:
labelencoder = LabelEncoder()
datos['mode'] = labelencoder.fit_transform(datos['mode'])
datos['key'] = labelencoder.fit_transform(datos['key'])


Codificamos la clase para poder utilizar el algoritmo de redes neuronales en el siguiente paso

In [ ]:
datos['music_genre'] = labelencoder.fit_transform(datos['music_genre'])

Visualizamos el dataset convertido

In [ ]:
datos.head(5)

Generamos 2 datasets: 1 con los atributos y otro unicamente con la clase

In [ ]:
X = datos.drop("music_genre",axis=1)
y = datos["music_genre"]

Dividimos el dataset de los atributos ("X") en uno de entrenamiento y otro de test

In [ ]:
porcentaje = 0.2
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=porcentaje,random_state=1)

Normalizamos el dataset Para ello usamos la función **min_max_scaler** que nos devuelve valores entre 0 y 1

In [ ]:
min_max_scaler = MinMaxScaler()
x_train_normalized = min_max_scaler.fit_transform(x_train)
x_test_normalized = min_max_scaler.fit_transform(x_test)

### Entrenamiento y validación de modelos
#### Modelo 1: Random Forest

Definimos los valores de los hiperparámetros y construimos el modelo

In [ ]:
n_estimators = 150
criterion = "gini"
max_depth = 10

In [ ]:
model = RandomForestClassifier(n_estimators = n_estimators, criterion=criterion,random_state=1,max_depth=max_depth)

Se entrena el modelo

In [ ]:
model.fit(x_train_normalized, y_train)

Validamos el modelo usando los datos de test

In [ ]:
y_predRF=model.predict(x_test_normalized)

Finalmente, se calcula el **accuracy** del modelo

In [ ]:
print("ACCURACY RANDOM FOREST:")
accuracy_score(y_test, y_predRF)                               

Se calculan otras métricas para hacer el análisis por clase y se dibuja la matriz de confusión

In [ ]:
print(classification_report(y_test, y_predRF))

In [ ]:
class_labels = ["Alternative","Anime","Blues","Classical","Country","Electronic","Hip-Hop","Jazz","Rap","Rock"]          
plt.figure()
ax= plt.subplot()
sns.heatmap(confusion_matrix(y_test, y_predRF), annot=True, fmt='g', ax=ax, cmap="Blues")
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels')
ax.set_title('Matriz de confusión para Algoritmo Random Forest')
ax.xaxis.set_ticklabels(class_labels,rotation=90); ax.yaxis.set_ticklabels(class_labels,rotation=0)

#### Modelo 2: Redes Neuronales

Definimos los valores de los hiperparámetros:

In [ ]:
epochs = 50
nodos_capa1 = 128
nodos_capa2 = 128

Construimos el modelo de redes neuronales con 3 capas profundas y una capa de salida

In [ ]:
# Configura semilla aleatoria
tf.random.set_seed(1)
modelRN = keras.Sequential([
    keras.layers.Dense(nodos_capa1,input_shape=[13],activation='relu'),
    keras.layers.Dense(nodos_capa2,activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
modelRN.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Realizamos una pequeña validación para comprobar la correcta construcción del modelo

In [ ]:
example_batch = x_train_normalized[:10]
example_result = modelRN.predict(example_batch)
example_result

Descripción del modelo

In [ ]:
modelRN.summary()

Entrenamos el modelo y calculamos el **accuracy** correspondiente

In [ ]:
modelRN.fit(x_train_normalized, y_train, epochs=epochs,verbose=0)

In [ ]:
test_loss, test_acc = modelRN.evaluate(x_test_normalized,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

Realizamos las predicciones usando los datos de test

In [ ]:
predictions = modelRN.predict(x_test_normalized)

In [ ]:
#Debido a que devuelve una probabilidad, se recupera el valor de la clase para poder calcular el accuracy y la matriz de confusión
ypredRN = predictions.argmax(axis=1)

In [ ]:
print("ACCURACY REDES NEURONALES:")
accuracy_score(y_test, ypredRN)

Obtenemos el resto de métricas y dibujamos la matriz de confusión

In [ ]:
print(classification_report(y_test, ypredRN))

In [ ]:
#confusion_matrix
plt.figure()
ax= plt.subplot()
sns.heatmap(confusion_matrix(y_test, ypredRN), annot=True, fmt='g', ax=ax, cmap="Blues")
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels')
ax.set_title('Matriz de confusión para algoritmo de redes neuronales')
ax.xaxis.set_ticklabels(class_labels,rotation=90); ax.yaxis.set_ticklabels(class_labels,rotation=0)